# Blood Pressure Analysis and Prediction
## Josh McComack
### Objectives
I wish to investigate the feasibility of predicting systolic and diastolic blood pressure based on the [National Health and Nutrition Examination Survey](https://www.kaggle.com/cdc/national-health-and-nutrition-examination-survey/home) (NHANES) data set from 2013-2014 found on kaggle.com. The primary questions I wish to answer are:
1. Which variables from the survey are most predictive empirically and do they correspond to the what mainstream literature identifies as key factors in blood pressure levels?
2. Comparing scikit-learn’s SGDRegressor, MultiTaskLasso, and RandomForestRegressor, which regression model offers the best predictions on this data set?
3. Does the best model perform well enough to serve as a possible supplementary or alternative way of “measuring” blood pressure?


### Import Libraries and Data
First we will import our libraries and data. The NHANES data is stored in 6 seperate csv files.

In [53]:
# Import libraries
import pandas as pd

In [54]:
# Load Data
path = "./national-health-and-nutrition-examination-survey/"
demographic_df = pd.read_csv(path + "demographic.csv")
diet_df = pd.read_csv(path + "diet.csv")
exam_df = pd.read_csv(path + "examination.csv")
labs_df = pd.read_csv(path + "labs.csv")
# There seems to be a non UTF-8 Character in the medications data set.
meds_df = pd.read_csv(path + "medications.csv", encoding = "latin1")
questionnaire_df = pd.read_csv(path + "questionnaire.csv")

In [55]:
# Join all data frames into a single data frame
combined_df = demographic_df
for frame in [diet_df, exam_df, labs_df, meds_df, questionnaire_df]:
    combined_df = combined_df.merge(frame, left_on="SEQN", right_on="SEQN", how="outer")
combined_df.head(10)

,SEQN,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDAGEMN,RIDRETH1,RIDRETH3,RIDEXMON,RIDEXAGM,...,WHD080U,WHD080L,WHD110,WHD120,WHD130,WHD140,WHQ150,WHQ030M,WHQ500,WHQ520
0,73557,8,2,1,69,NaN,4,4,1.0,NaN,...,NaN,40.0,270.0,200.0,69.0,270.0,62.0,NaN,NaN,NaN
1,73557,8,2,1,69,NaN,4,4,1.0,NaN,...,NaN,40.0,270.0,200.0,69.0,270.0,62.0,NaN,NaN,NaN
2,73558,8,2,1,54,NaN,3,3,1.0,NaN,...,NaN,NaN,240.0,250.0,72.0,250.0,25.0,NaN,NaN,NaN
3,73558,8,2,1,54,NaN,3,3,1.0,NaN,...,NaN,NaN,240.0,250.0,72.0,250.0,25.0,NaN,NaN,NaN
4,73558,8,2,1,54,NaN,3,3,1.0,NaN,...,NaN,NaN,240.0,250.0,72.0,250.0,25.0,NaN,NaN,NaN
5,73558,8,2,1,54,NaN,3,3,1.0,NaN,...,NaN,NaN,240.0,250.0,72.0,250.0,25.0,NaN,NaN,NaN
6,73559,8,2,1,72,NaN,3,3,2.0,NaN,...,NaN,NaN,180.0,190.0,70.0,228.0,35.0,NaN,NaN,NaN
7,73559,8,2,1,72,NaN,3,3,2.0,NaN,...,NaN,NaN,180.0,190.0,70.0,228.0,35.0,NaN,NaN,NaN
8,73559,8,2,1,72,NaN,3,3,2.0,NaN,...,NaN,NaN,180.0,190.0,70.0,228.0,35.0,NaN,NaN,NaN
9,73559,8,2,1,72,NaN,3,3,2.0,NaN,...,NaN,NaN,180.0,190.0,70.0,228.0,35.0,NaN,NaN,NaN


### Data Selection
Our data set contains a total 1824 different columns, with overview of the columns found [here](https://www.kaggle.com/cdc/national-health-and-nutrition-examination-survey/home). To get started however, we will focus on features which have been identified as possible factors in blood presssure levels:
* Age [1][2]
* Gender [2]
* Sodium intake [1]
* Smoking [1][2]
* Exercise levels [1][2]
* Alcohol intake [1][2]
* Diabetes [2]
* Weight or Obesity [1][2]

* Family history of high blood pressure [1][2]
* Genetics (Which probably ties in with gender, and family history) [1]
* Stress levels [1][2]
* Chronic kidney Disease [1][2]


References:
* [1] [www.webmd.com](https://www.webmd.com/hypertension-high-blood-pressure/guide/blood-pressure-causes#1)
* [2] [www.heart.org](http://www.heart.org/en/health-topics/high-blood-pressure/why-high-blood-pressure-is-a-silent-killer/know-your-risk-factors-for-high-blood-pressure)


In [56]:
# We will create a list to store all of our selected columns for convenience. 
selected_columns = [];

From the demographic data set, we can use the survey participants' age and gender identified in columns DMDHRAGE and RIAGENDR defined in the detailed column description found [here](https://wwwn.cdc.gov/Nchs/Nhanes/Search/variablelist.aspx?Component=Demographics&CycleBeginYear=2013).

In [57]:
demographic_columns = ['DMDHRAGE', 'RIAGENDR']
selected_columns += demographic_columns
demographic_df[demographic_columns].head()

,DMDHRAGE,RIAGENDR
0,69,1
1,54,1
2,72,1
3,33,1
4,78,2


The dietary data set contains potentially usefull information about sodium intake and diet, however the detailed column description does not explain how this information is encoded. So we will use the more quantifiable sodium column LBXSNASI taken from the [laboratory data set](https://wwwn.cdc.gov/Nchs/Nhanes/Search/variablelist.aspx?Component=Laboratory&CycleBeginYear=2013). 

Note: there seems to be many more sodium columns missing this data set.

In [58]:
labs_columns = ['LBXSNASI']
selected_columns += labs_columns
labs_df[labs_columns].head()

,LBXSNASI
0,136.0
1,128.0
2,142.0
3,NaN
4,142.0


The [questionnaire](https://wwwn.cdc.gov/Nchs/Nhanes/Search/variablelist.aspx?Component=Questionnaire&CycleBeginYear=2013) contains lots of varius data about smoking, exercise, and alcohol habits. A few columns we will use are:
##### Smoking

* SMQ040: Do you now smoke cigarettes?
* SMQ020: Have you smoked at least 100 cigarettes in your entire life?
* SMD650: During the past 30 days, on the days that you smoked, about how many cigarettes did you smoke per day?
* SMD641: On how many of the past 30 days did you smoke a cigarette?

##### Exercise

* PAQ677: On how many of the past 7 days did you exercise or participate in physical activity for at least 20 minutes that made you sweat and breathe hard, such as basketball, soccer, running, swimming laps, fast bicycling, fast dancing, or similar activities?
* PAQ678: On how many of the past 7 days did you do exercises to strengthen or tone your muscles, such as push-ups, sit-ups, or weight lifting?

##### Alcohol

* ALQ101: In any one year, have you had at least 12 drinks of any type of alcoholic beverage? By a drink, I mean a 12 oz. beer, a 5 oz. glass of wine, or one and half ounces of liquor.
* ALQ130: In the past 12 months, on those days that you drank alcoholic beverages, on the average, how many drinks did you have?

##### Diabetes
* DIQ010: Other than during pregnancy, have you ever been told by a doctor or health professional that you have diabetes or sugar diabetes?

In [59]:
questionnaire_columns = ['SMQ040', 'SMQ020', 'SMD650', 'SMD641', 'PAQ677', 'PAQ678', 'ALQ101', 'ALQ130', 'DIQ010']
selected_columns += questionnaire_columns
questionnaire_df[questionnaire_columns].head()

,SMQ040,SMQ020,SMD650,SMD641,PAQ677,PAQ678,ALQ101,ALQ130,DIQ010
0,3.0,1.0,NaN,NaN,NaN,NaN,1.0,1.0,1.0
1,2.0,1.0,1.0,1.0,NaN,NaN,1.0,4.0,1.0
2,3.0,1.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
4,NaN,2.0,NaN,NaN,NaN,NaN,1.0,NaN,2.0


Examining the medications data set, we can see who is taking medication for blood pressure and/or diabetes. 


In [60]:
meds_columns = ['RXDDRUG', 'RXDRSD1']
selected_columns += meds_columns
meds_df[meds_columns].head(11)

,RXDDRUG,RXDRSD1
0,99999,NaN
1,INSULIN,Type 2 diabetes mellitus
2,GABAPENTIN,Restless legs syndrome
3,INSULIN GLARGINE,Type 2 diabetes mellitus
4,OLMESARTAN,Type 2 diabetes mellitus with kidney complicat...
5,SIMVASTATIN,Pure hypercholesterolemia
6,INSULIN ASPART,Type 2 diabetes mellitus
7,INSULIN GLARGINE,Type 2 diabetes mellitus
8,PANCRELIPASE,"Disease of pancreas, unspecified"
9,SIMVASTATIN,Pure hypercholesterolemia


The data set doesn't seem to contain direct information on stress levels, chronic kidney disease, or family history of high blood pressure. But from the examination data set we can collect information on body mass index and blood pressure readings:

#####  Body Mass Index

* BMXBMI: Body Mass Index (kg/m**2)	

#####  Body Mass Index
* BPXDI 1-4: Diastolic blood pressure mm Hg (first through fourth readings)
* BPXSY 1-4: Systolic blood pressure mm Hg (first through fourth readings)

In [61]:
exam_columns = ['BMXBMI', 'BPXDI1', 'BPXDI2', 'BPXDI3', 'BPXDI4', 'BPXSY1', 'BPXSY2', 'BPXSY3', 'BPXSY4']
selected_columns += exam_columns
exam_df[exam_columns].head()

,BMXBMI,BPXDI1,BPXDI2,BPXDI3,BPXDI4,BPXSY1,BPXSY2,BPXSY3,BPXSY4
0,26.7,72.0,76.0,74.0,NaN,122.0,114.0,102.0,NaN
1,28.6,62.0,80.0,42.0,NaN,156.0,160.0,156.0,NaN
2,28.9,90.0,76.0,80.0,NaN,140.0,140.0,146.0,NaN
3,17.1,38.0,34.0,38.0,NaN,108.0,102.0,104.0,NaN
4,19.7,86.0,88.0,86.0,NaN,136.0,134.0,142.0,NaN


### Data Cleaning
Now that we've identified the columns we wish to use, lets drop them into a single data frame.

In [64]:
selected_df = combined_df[selected_columns]
selected_df.head()

,DMDHRAGE,RIAGENDR,LBXSNASI,SMQ040,SMQ020,SMD650,SMD641,PAQ677,PAQ678,ALQ101,...,RXDRSD1,BMXBMI,BPXDI1,BPXDI2,BPXDI3,BPXDI4,BPXSY1,BPXSY2,BPXSY3,BPXSY4
0,69,1,136.0,3.0,1.0,NaN,NaN,NaN,NaN,1.0,...,NaN,26.7,72.0,76.0,74.0,NaN,122.0,114.0,102.0,NaN
1,69,1,136.0,3.0,1.0,NaN,NaN,NaN,NaN,1.0,...,Type 2 diabetes mellitus,26.7,72.0,76.0,74.0,NaN,122.0,114.0,102.0,NaN
2,54,1,128.0,2.0,1.0,1.0,1.0,NaN,NaN,1.0,...,Restless legs syndrome,28.6,62.0,80.0,42.0,NaN,156.0,160.0,156.0,NaN
3,54,1,128.0,2.0,1.0,1.0,1.0,NaN,NaN,1.0,...,Type 2 diabetes mellitus,28.6,62.0,80.0,42.0,NaN,156.0,160.0,156.0,NaN
4,54,1,128.0,2.0,1.0,1.0,1.0,NaN,NaN,1.0,...,Type 2 diabetes mellitus with kidney complicat...,28.6,62.0,80.0,42.0,NaN,156.0,160.0,156.0,NaN


In [ ]:
def print_uniques(df):
    """Prints each unique value for each column in the data frame 
       with a null count.
    """
    for column in df.columns:
        print('{} ({}):{}\n'.format(column, df[column].dtypes, set(df[column].astype('str'))))